In [1]:
import json_lines
import pandas as pd
import pickle
import numpy as np

import lxml.etree as etree # to create gexl file
import datetime

import geopandas
import geopy

import itertools

In [1]:
import pandas as pd
import pickle
import numpy as np
import itertools


In [ ]:
# Define function to extract tweet and retweet information

def load_jsonl(file):
    tweets = []
    with open(file, 'rb') as f:
        for tweet in json_lines.reader(f, broken=True):             
            if 'retweeted_status' in tweet:           
                reduced_tweet = {'retweeting_username': tweet['user']['screen_name'],
                                 'retweeted_username' : tweet['retweeted_status']['user']['screen_name']}          
                tweets.append(reduced_tweet)            
        return (tweets)

In [ ]:
# Extract all retweets from large (40GB) jsonl files
#retweets_all = load_jsonl(r"D:\twarc\climatetweetID\sna\tweets_small.jsonl")
retweets_all = load_jsonl(r"D:\twarc\climatetweetID\hydratefile\climate_id_03.jsonl")
with open(r"D:\twarc\climatetweetID\sna\coretweet_network\01_all_retweets\all_retweet_3.txt", "wb") as fp:   #Pickling
        pickle.dump(retweets_all, fp)

In [2]:
# top influential user's name
#top_users = pd.read_csv(r"D:\twarc\climatetweetID\tweets_prediction\influence\allusers_info\output_3\top1000.csv", dtype = object)      
top_users = pd.read_csv(r"E:\twarc\climatetweetID\tweets_prediction\influence\allusers_info\output_3\top5000_uslabelled.csv", dtype = object)
top_users = top_users.dropna(subset = ['NAME'])
user_name = top_users['username']
user_name = user_name[:1200]

In [3]:
# keep only influencers' retweets
# https://www.jianshu.com/p/a3f60ceff8bf
# https://stackoverflow.com/questions/16597265/appending-to-an-empty-dataframe-in-pandas
for i in range(0,4):    
    with open(r"E:\twarc\climatetweetID\sna\coretweet_network\01_all_retweets\all_retweet_%d.txt" % (i), "rb") as fp:   #Pickling
        tweets = pickle.load(fp)        
    tweets = pd.DataFrame(tweets)
    infl_tweet = pd.DataFrame()
    for name in user_name.tolist():
        subset = tweets[tweets['retweeted_username'] == name]
        infl_tweet = infl_tweet.append(subset)    
    with open(r"E:\twarc\climatetweetID\sna\coretweet_network\02_influencer_retweets\infl_retweet_1200_%d.txt" % (i), "wb") as fp:
        pickle.dump(infl_tweet, fp)        

In [4]:
# group by influencer and retweeting users
infl_tweet = pd.DataFrame()
for i in range(0,4):
    with open(r"E:\twarc\climatetweetID\sna\coretweet_network\02_influencer_retweets\infl_retweet_%d.txt" % (i), "rb") as fp:   #Pickling
        subset = pickle.load(fp)        
    infl_tweet = infl_tweet.append(subset)    
retweet_summary = infl_tweet.groupby(['retweeting_username','retweeted_username']).size()
retweet_summary = pd.DataFrame(retweet_summary)
retweet_summary.reset_index(inplace = True)
retweet_summary = retweet_summary.rename(columns = {0: "sum_tweets"})

In [8]:
# build the co-retweet matrix!
# https://stackoverflow.com/questions/42806398/create-adjacency-matrix-for-two-columns-in-pandas-dataframe
# https://stackoverflow.com/questions/51217328/turn-pandas-dataframe-into-int32-numpy-matrix

df = pd.crosstab(retweet_summary.retweeting_username, retweet_summary.retweeted_username)
#df.to_csv(r"D:\twarc\climatetweetID\sna\coretweet_network\03_coretweet_matrix\coretweet.csv")
#df = pd.read_csv(r"D:\twarc\climatetweetID\sna\coretweet_network\03_coretweet_matrix\coretweet.csv")
re_mat = np.asmatrix(df.values)
co_mat = re_mat.T*re_mat
#pd.DataFrame(co_mat).to_csv(r"D:\twarc\climatetweetID\sna\coretweet_network\03_coretweet_matrix\co_mat.csv")
column_name =  list(df.columns)
df_comat = pd.DataFrame(co_mat)
df_comat.columns = column_name
df_comat.index = column_name

In [12]:
# influencer's name and retweeting user's name
# export Gephi files
pair_user = list(itertools.combinations(column_name, 2))
co_retweet_matrix = []
for i in pair_user:
    no_coretweet = df_comat.at[i[0], i[1]]
    new_pair = [i[0], i[1], no_coretweet]    
    co_retweet_matrix.append(new_pair)
co_retweet_matrix = pd.DataFrame(co_retweet_matrix, columns = ['userA', 'userB', 'no_coretweet'])
co_retweet_matrix.to_csv(r"E:\twarc\climatetweetID\sna\coretweet_network\04_Gephi\edge_table.csv")

In [65]:
co_retweet_matrix


,userA,userB,no_coretweet
0,1followernodad,4everNeverTrump,21
1,1followernodad,ABC,17
2,1followernodad,ABCPolitics,2
3,1followernodad,AOC,222
4,1followernodad,AP_Politics,12
...,...,...,...
76631,washingtonpost,weatherchannel,634
76632,washingtonpost,wesley_jordan,655
76633,wattsupwiththat,weatherchannel,5
76634,wattsupwiththat,wesley_jordan,1


In [70]:
def create_gexf(tweets, filename):
    attr_qname = etree.QName("http://www.w3.org/2001/XMLSchema-instance", "schemaLocation")
    gexf = etree.Element('gexf',
                         {attr_qname : 'http://www.gexf.net/1.3draft  http://www.gexf.net/1.3draft/gexf.xsd'},
                         nsmap={None : 'http://graphml.graphdrawing.org/xmlns/graphml'},
                         version = '1.3')
    graph = etree.SubElement(gexf,'graph', defaultedgetype = 'undirected', mode =  'dynamic', timeformat='datetime')
    attributes = etree.SubElement(graph, 'attributes', {'class' : 'node', 'mode' : 'static'})  
    
    nodes = etree.SubElement(graph, 'nodes')
    edges = etree.SubElement(graph, 'edges')

    for index, tweet in tweets.iterrows():
        node = etree.SubElement(nodes,'node', id = tweet['userA'], Label = tweet['userA'])
        node = etree.SubElement(nodes,'node', id = tweet['userB'], Label = tweet['userB'])
        etree.SubElement(edges, 'edge',
                         {'source' : tweet['userA'],
                          'target' : tweet['userB'],
                          'no_cort': str(tweet['no_coretweet'])})
        

    with open(filename, 'w', encoding='utf-8')as f:
        f.write(etree.tostring(gexf, encoding='utf8', method='xml').decode('utf-8'))
    print('Created gexf.')

In [71]:
create_gexf(co_retweet_matrix, r'D:\twarc\climatetweetID\sna\coretweet_network\03_coretweet_matrix\full_test_network.gexf')

Created gexf.


In [79]:
nodetable = pd.read_csv(r"D:\twarc\climatetweetID\sna\coretweet_network\04_Gephi\nodetable_o.csv")
node_newatt = pd.read_csv(r"D:\twarc\climatetweetID\sna\coretweet_network\04_Gephi\topuser_info.csv")
nodetable.set_index('Id').join(node_newatt.set_index('username')).to_csv(r"D:\twarc\climatetweetID\sna\coretweet_network\04_Gephi\new_nodetable.csv")